In [8]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

# 讀取數據
test_data = np.load('Data/test.npy')

def resize_data(X_train, new_len):

    if new_len > X_train.shape[1]:
        print(f"new_len {new_len} is greater than the current length {X_train.shape[1]}, returning original dataset.")
        return X_train
    
    X_resized = X_train[:, :new_len]
    
    return X_resized

# 創建標籤
test_label = np.zeros(len(test_data))

# 合併數據和標籤
X_test = resize_data(test_data, 10240)
print(X_test.shape)
print(test_label.shape)
X_train_tensor = torch.tensor(X_test, dtype=torch.float).unsqueeze(1)  # Adding channel dimension
y_train_tensor = torch.tensor(test_label, dtype=torch.long)  # Assuming classification task
test_loader = DataLoader(X_train_tensor, shuffle=False)
for ii in test_loader:
    print(ii.shape)
    break


(240, 10240)
(240,)
torch.Size([1, 1, 10240])


In [9]:
label_table = {
    0: 'normal',
    1: 'outer_break',
    2: 'inner_break',
}

In [15]:
def test(model, test_loader):
    model.eval()
    model = model.cuda()
    answer = []
    for inputs in tqdm(test_loader):
        inputs = inputs.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        # print(predicted.item())
        answer.append(predicted.item())
    return answer

In [16]:
model = torch.jit.load('model_27.pt')
print(model)
answer = test(model, test_loader)

RecursiveScriptModule(
  original_name=mod_resnet
  (conv1): RecursiveScriptModule(original_name=Conv1d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm1d)
  (maxpool): RecursiveScriptModule(original_name=MaxPool1d)
  (avgpool): RecursiveScriptModule(original_name=AdaptiveAvgPool1d)
  (fc): RecursiveScriptModule(original_name=Linear)
)


100%|██████████| 240/240 [00:00<00:00, 3072.32it/s]


In [19]:
import csv

file_name = 'HW3.csv'

with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    for index in answer:
        item = label_table[index]
        writer.writerow([item])  # Write each item in its own row
